In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
path = '/home/manoelflorencio/cta_for_jd/LakeBench'
os.chdir(path)
print(os.getcwd())

/home/manoelflorencio/cta_for_jd/LakeBench


In [4]:
def generate_keys(df):
    keys = []
    for i in range(df.shape[0]):
        key = df.iloc[i, 0] + '#' + df.iloc[i, 1]
        keys.append(key)
    return keys

In [5]:
def sorting_tablenames(df):
    sorted_df = pd.DataFrame()
    for i in range(df.shape[0]):
        ds1 = df.iloc[i,0]
        ds2 = df.iloc[i,1] 
    
        ds_esq, ds_dir = '', ''
        
        if(ds1 < ds2):
            ds_esq, ds_dir = ds1, ds2
        else:
            ds_esq, ds_dir = ds2, ds1
        
        if(len(sorted_df) == 0):
            sorted_df = pd.DataFrame({'LEFT_TABLE': [ds_esq+''], 'RIGHT_TABLE': [ds_dir]})
        else:
            sorted_df = pd.concat([sorted_df, pd.DataFrame({'LEFT_TABLE': [ds_esq], 'RIGHT_TABLE': [ds_dir]})], axis=0)

    sorted_df = sorted_df.drop_duplicates()
    sorted_df = sorted_df.reset_index(drop=True)

    return sorted_df

In [6]:
groundTruth = pd.read_csv('opendata_join_ground_truth_clean.csv')
groundTruth.head()

,query_table,candidate_table,query_column,candidate_column
0,SG_CSV0000000000000925.csv,SG_CSV0000000000001714.csv,ban_education4,ban_education4
1,SG_CSV0000000000000925.csv,SG_CSV0000000000001714.csv,levelsch,levelsch
2,SG_CSV0000000000000925.csv,SG_CSV0000000000001714.csv,socialnetwork_5_slice,socialnetwork_5_slice
3,SG_CSV0000000000000925.csv,SG_CSV0000000000001714.csv,ban_children4,ban_children4
4,SG_CSV0000000000000925.csv,SG_CSV0000000000001714.csv,q20_q212,q20_q212


In [7]:
groundTruth_table = groundTruth[['query_table','candidate_table']]
groundTruth_table = groundTruth_table.drop_duplicates()
groundTruth_table.head()

,query_table,candidate_table
0,SG_CSV0000000000000925.csv,SG_CSV0000000000001714.csv


In [17]:
jd_df = pd.read_csv('table_cartesians_JD_with_columns_descriptions.csv')
jd_df_925 = pd.read_csv('table_cartesians_JD_with_columns_descriptions_SG_CSV0000000000000925.csv')
jd_df = pd.concat([jd_df, jd_df_925])
jd_df = jd_df.drop_duplicates()
jd_df = jd_df.reset_index(drop=True)

jd_df = jd_df[jd_df['JOINABLE'] == 'Yes']
jd_df = jd_df[['LEFT_TABLE','RIGHT_TABLE']]
jd_df = sorting_tablenames(jd_df)
jd_df.head()

,LEFT_TABLE,RIGHT_TABLE
0,SG_CSV0000000000000744.csv,SG_CSV0000000000001178.csv
1,SG_CSV0000000000001178.csv,SG_CSV0000000000001674.csv
2,SG_CSV0000000000001175.csv,SG_CSV0000000000001178.csv
3,SG_CSV0000000000001178.csv,SG_CSV0000000000001851.csv
4,SG_CSV0000000000000577.csv,SG_CSV0000000000001178.csv


In [18]:
jd_df.shape

(761, 2)

In [19]:
groundTruth_table['KEY'] = generate_keys(groundTruth_table)
jd_df['KEY'] = generate_keys(jd_df)

In [20]:
groundTruth_table

,query_table,candidate_table,KEY
0,SG_CSV0000000000000925.csv,SG_CSV0000000000001714.csv,SG_CSV0000000000000925.csv#SG_CSV0000000000001...


In [13]:
groundTruth_table.shape

(1, 3)

In [22]:
jd_df.shape

(761, 3)

In [23]:
tp = 0
fp = 0
fn = 0

for i in range(len(jd_df)):
    if(jd_df.iloc[i,2] in groundTruth_table.iloc[:,2].values):
        tp += 1
    else:
        fp += 1
    
for i in range(len(groundTruth_table)):
    if(groundTruth_table.iloc[i,2] not in jd_df.iloc[:,2].values):
        fn += 1
    
precision = (tp)/(tp+fp)
recall = (tp)/(tp+fn)
f1_score = 2 * (precision*recall)/(precision+recall)

print(precision, recall, f1_score)

0.001314060446780552 1.0 0.0026246719160104987


# Falsos positivos

In [24]:
for i in range(len(jd_df)):
    if(jd_df.iloc[i,2] not in groundTruth.iloc[:,2].values):
        print(jd_df.iloc[i,2])

SG_CSV0000000000000744.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000001178.csv#SG_CSV0000000000001674.csv
SG_CSV0000000000001175.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000001178.csv#SG_CSV0000000000001851.csv
SG_CSV0000000000000577.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000000571.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000000918.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000000099.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000000360.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000000446.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000000119.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000000299.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000000358.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000001178.csv#SG_CSV0000000000001789.csv
SG_CSV0000000000000460.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000000128.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000001178.csv#SG_CSV0000000000001738.csv
SG_CSV0000000000001177.csv#SG_CSV0000000000001178.csv
SG_CSV0000000000000578.csv#S

# Falsos Negativos

In [ ]:
for i in range(len(groundTruth)):
    if(groundTruth.iloc[i,2] not in jd_df.iloc[:,2].values):
        print(groundTruth.iloc[i,2])